# Introduction

This notebook aims to provide users convenient visualization tools after Anomaly Detection, and inform how the time-series go and if the MVAD works well. 
Visualization takes place within Synapse, are made interactive and wrapped up in one-piece so users do not have to switch to ADX or make extra efforts on ADX queries and plots. 

Specifically, the notebook connects the ADX resulting table and visualize the following for user insights:

- Check time-series for pattern and surges and dips.
- Highlight where the anomalies are detected and show whether the decision was correct in distinct colored regions with ground truth.
- Show severity- how confident the algorithm was to announce anomalies, so customers can adjust this important parameter to catch anomalies as much as possible with minimum noises.
- For each triggered anomaly indicate the top telemetry contributors as potential root causes.
- Concrete numbers of popular metrics for classification as performance assessment, namely, confusion matrix, f1, precision and recall.

### Section 1. Import modules and define functions

In [ ]:
import os
import sys
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from datetime import datetime as dt
import pytz
import ast
from pathlib import Path

In [ ]:
def query_adx_data(adx_linked_service, adx_database, query):
    """ Query ADT data and return the data as Spark dataframe
    :param adx_linked_service: name of the ADX (historized data store) linked service registered in Synapse workspace
    :type: string

    :param adx_database: ADX database name containing historized ADX data
    :type: string

    :param query: ADT-ADX joint query
    :type: string

    :return: dataframe containing queried data
    :type: Spark dataframe
    """
    df  = spark.read \
        .format("com.microsoft.kusto.spark.synapse.datasource") \
        .option("spark.synapse.linkedService", adx_linked_service) \
        .option("kustoDatabase", adx_database) \
        .option("kustoQuery", query) \
        .option("authType", "LS") \
        .load()
    return df


def get_mvad_result_from_adx(
    cols_to_include=None, time_window=None,
    adx_linked_service=None, adx_database=None, adx_table=None) -> pd.DataFrame:
    """
    Fetch data with MVAD results from ADX.

    Parameters
    ----------
    cols_to_include : columns to include for plots,
        list of str (e.g. ['dtmi_syntheticfactory_sourcemachine2_1_A_Amps_Ia', 
                           'dtmi_syntheticfactory_sourcemachine2_1_A_Amps_Ib',
                           'dtmi_syntheticfactory_sourcemachine2_1_A_Amps_Ic'])
    time_window : indicate whether to save the topology json file,
        list of str (e.g. '2022-04-19 04:26:00', '2022-04-19 09:00:00'])
    adx_linked_service : name of ADX linkedService,
        str
    adx_database : name of ADX Database,
        str
    adx_table : name of ADX MVAD resulting Table,
        str

    Return
    ----------
    df : MVAD inference results with specified columns in certain time window,
        pd.DataFrame
    """
    adx_query = f"{adx_table}" 
    if cols_to_include is not None:
        adx_query = f"{adx_table} | project timestamp, result, isAnomaly"
        for col in cols_to_include:
            adx_query += f", {col}"
    if time_window is not None:
        start_time = pytz.utc.localize(dt.strptime(time_window[0], '%Y-%m-%d %H:%M:%S'))
        end_time = pytz.utc.localize(dt.strptime(time_window[1], '%Y-%m-%d %H:%M:%S'))
        adx_query += f" | where todatetime(timestamp)>=datetime({start_time}) and todatetime(timestamp)<=datetime({end_time})"
        
    df = query_adx_data(adx_linked_service, adx_database, adx_query)
    
    df = df.toPandas()
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df = df.sort_values('timestamp').reset_index(drop=True).drop('isAnomaly', axis=1)
    df['result'] = df['result'].str.replace('false', 'False')
    df['result'] = df['result'].str.replace('true', 'True')
    df['result'] = df['result'].apply(ast.literal_eval)
    df = pd.concat([df, pd.json_normalize(df['result'])], axis=1)
    df = df.drop('result', axis=1)
    
    features = cols_to_include if cols_to_include is not None else [col for col in df.columns if col not in ['timestamp', 'isAnomaly', 'score', 'severity', 'contributors']]
    empty_contributor_dic = {}
    for i in range(len(features)):
        empty_contributor_dic['series_'+str(i)]=0

    if 'contributors' in df.columns:
        df['contributors'] = df['contributors'].apply(lambda x: parse_contributors(x, empty_contributor_dic))
        df = pd.concat([df.drop(['contributors'], axis=1), pd.json_normalize(df['contributors'])], axis=1)
        df = df[['timestamp', 'isAnomaly', 'score', 'severity'] + features + [f'series_{i}' for i in range(len(features))]] # Hardcoded until MVAD fix the series naming issue
    else:
        df = df[['timestamp', 'isAnomaly', 'score', 'severity'] + features]
        for i in range(len(features)):
            df[f'series_{i}'] = 0
    return df


def parse_contributors(x=None, empty_contributor_dic=None) -> dict:
    """
    Parse the raw contributors json into clean dict format.

    Parameters
    ----------
    x : raw contributors json to parse,
        list of dict
    empty_contributor_dic : for normal situation, return an empty contributor dict with all contributionScore equal zero,
        dict

    Return
    ----------
    parsed_contributors_dic : parsed contributor dict in the format of {Contributor1: contributionScore1, Contributor2: contributionScore2, ...},
        dict
    """    
    if x!=x:
        return empty_contributor_dic
    parsed_contributors_dic = {}
    for tmp_dic in x:
        parsed_contributors_dic[tmp_dic['variable']] = tmp_dic['contributionScore']
    return parsed_contributors_dic


def plot_mvad(df=None, cols_to_plot=None, min_severity=None, label=None) -> None:
    """
    Make the following three MVAD results plots:
    1. If labels not given, plot the Time-series with Anomalies Detected; 
       Otherwise if label is provided, plot MVAD performance that filled with colored regions denote TN/FN/FP/TP/Label NA;
    2. Severity of Anomalies;
    3. Contribution to Anomalies.

    Parameters
    ----------
    df : dataframe fetched from ADX to plot,
        pd.DataFrame
    cols_to_plot : list of columns to include in the plots,
        list of str
    min_severity : set the minimum severity threshold to define an anomaly,
        float
    label : label dataframe to evaluate and plot MVAD performance, with at least two columns of 'Timestamp' and 'isAnomaly',
        pd.DataFrame, default=None

    Return
    ----------
    None
    """
    df_plot = df.copy()
    tmp_df_plot_anomaly = df_plot[df_plot['isAnomaly']==True]
    df_plot['isAnomaly'] = False
    
    severity_thres_default = tmp_df_plot_anomaly['severity'].min() if tmp_df_plot_anomaly.shape[0]>0 else 1
    severity_thres = min_severity or severity_thres_default
    df_plot.loc[df_plot['severity']>=severity_thres, 'isAnomaly'] = True
    severity_thres = round(severity_thres, 3)

    fig1_title = f'Time-series Plot with Anomalies Detected (Magenta) with min_severity={severity_thres}' if label is None \
        else f'MVAD Performance Plot with min_severity={severity_thres}<br>- Green: TN|Yellow: FN|Blue: FP|Red: TP|Grey: Label NA'
    fig1 = make_subplots(subplot_titles=[fig1_title])
    for feature in cols_to_plot:
        fig1.add_trace(go.Scatter(x=df_plot['timestamp'], y=df_plot[feature], name=feature, mode='lines'), row=1, col=1)

    ymax = df_plot[cols_to_plot].max().max()
    ymin = df_plot[cols_to_plot].min().min()
    update_layout_shapes = []

    if label is None:
        for i in df_plot[df_plot['isAnomaly']==1].index:
            if i!=df_plot.shape[0]-1:
                update_layout_shapes.append(dict(type='rect',
                                                 xref='x',
                                                 yref='y',
                                                 x0=str(df_plot.iloc[i]['timestamp']),
                                                 y0=str(ymin),
                                                 x1=str(df_plot.iloc[i+1]['timestamp']),
                                                 y1=str(ymax),
                                                 fillcolor='magenta',
                                                 layer='below',
                                                 opacity=1,
                                                 line_width=0))
        fig1.update_layout(shapes=update_layout_shapes)
    else:
        label = label.rename(columns={'Timestamp': 'timestamp', 'isAnomaly': 'isAnomaly_label'})
        label['timestamp'] = pd.to_datetime(label['timestamp']).dt.tz_localize('utc')
        label = label[['timestamp', 'isAnomaly_label']]
        
        tmp_df_plot = df_plot[['timestamp', 'isAnomaly']].sort_values('timestamp').reset_index(drop=True)
        tmp_df_plot = tmp_df_plot.rename(columns={'isAnomaly': 'isAnomaly_mvad'})
        tmp_df_plot_merge = tmp_df_plot.merge(label, how='left', on='timestamp')
        tmp_df_plot_merge['confusion'] = tmp_df_plot_merge.apply(
            lambda x: 'TP' if x['isAnomaly_mvad']==x['isAnomaly_label']==True
                else ('FP' if (x['isAnomaly_mvad']== True) and (x['isAnomaly_label']==False)
                else ('TN' if x['isAnomaly_mvad']==x['isAnomaly_label']==False 
                else ('FN' if (x['isAnomaly_mvad']== False) and (x['isAnomaly_label']==True) 
                else 'NA'))), axis=1)
        tmp_df_plot_merge = tmp_df_plot_merge.set_index('timestamp')
        

        for j in range(tmp_df_plot_merge.shape[0]-1):
            confusion = tmp_df_plot_merge.iloc[j]['confusion']
            if confusion == 'TN':
                fillcolor = 'green'
            elif confusion == 'FN':
                fillcolor = 'yellow'
            elif confusion == 'FP':
                fillcolor = 'blue'
            elif confusion == 'TP':
                fillcolor = 'red'
            else:
                fillcolor = 'grey'
            update_layout_shapes.append(dict(type='rect',
                                             xref='x',
                                             yref='y',
                                             x0=str(tmp_df_plot_merge.index[j]),
                                             y0=str(ymin),
                                             x1=str(tmp_df_plot_merge.index[j+1]),
                                             y1=str(ymax),
                                             fillcolor=fillcolor,
                                             layer='below',
                                             opacity=0.5,
                                             line_width=0))
        fig1.update_layout(shapes=update_layout_shapes)
    
    fig1.update_xaxes(title_text='Timestamp')
    fig1.update_yaxes(title_text='Value')
    fig1.show()
        
    fig2 = make_subplots(subplot_titles=[f'Severity of Anomalies with min_severity={severity_thres}'])
    fig2.add_trace(go.Scatter(x=df_plot['timestamp'], y=df_plot['severity'], 
                              name='Severity', mode='lines'))
    fig2.add_trace(go.Scatter(x=df_plot['timestamp'], y=[severity_thres]*df_plot.shape[0], 
                              name='min_severity', mode='lines', line=dict(dash='dash')))
    fig2.update_xaxes(title_text='Timestamp')
    fig2.update_yaxes(title_text='Value')
    fig2.show()

    fig3 = px.bar(df_plot.fillna(0), 
                  x='timestamp', y=[f'series_{i}' for i in range(len(cols_to_plot))], 
                  title='Contribution to Anomalies')
    fig3.update_xaxes(title_text='Timestamp')
    fig3.update_yaxes(title_text='Contribution Scores')
    fig3.update_layout(title_x=0.5)
    fig3.show()


def evaluate(label=None, df_res=None, \
             floor_bin='15min', severity_thres=None, print_out=True) -> pd.DataFrame:
    """
    Evaluate MVAD performance in terms of confusion matrix, f1, precision and recall.

    Parameters
    ----------
    label : label dataframe to evaluate MVAD performance, with at least two columns of 'timestamp' and 'isAnomaly',
        pd.DataFrame
    df_res : dataframe fetched from ADX to evaluate,
        pd.DataFrame
    floor_bin : the length of timestamp bins to aggregate AD results, 
        for example, if original results are made per minute and floor_bin is set to '15min',
        then the outcome of 15min-result is True if and only if any single minute of 15min before aggregation give a True, otherwise False,
        str, default='15min'
    severity_thres : set the minimum severity threshold to define an anomaly, taking the MVAD default if none specified,
        float
    print_out : indicate whether to print out confusion matrix and other metrics,
        bool, default=True

    Return
    ----------
    confusion_matrix : confusion matrix of TP, FP, TN and FN,
        pd.DataFrame
    """
    tmp_df_res = df_res[['timestamp', 'value.is_anomaly', 'value.severity']]
    tmp_df_res['timestamp'] = pd.to_datetime(tmp_df_res['timestamp']).dt.tz_localize(None)
    tmp_df_res = tmp_df_res.sort_values('timestamp').reset_index(drop=True)
    
    severity_thres = severity_thres or tmp_df_res[tmp_df_res['value.is_anomaly']]['value.severity'].min()
    tmp_df_res.loc[tmp_df_res['value.severity']<severity_thres, 'value.is_anomaly'] = False
    
    tmp_label = tmp_df_res[['timestamp']].merge(label, how='inner', on='timestamp')\
                          .sort_values('timestamp').reset_index(drop=True)

    tmp_df_res['timestamp_floor'] = tmp_df_res['timestamp'].dt.floor(floor_bin)
    df_res_eval = pd.DataFrame(tmp_df_res.groupby('timestamp_floor')['value.is_anomaly'].apply(lambda x: any(list(x))))
    tmp_label['timestamp_floor'] = tmp_label['timestamp'].dt.floor(floor_bin)
    label_eval = pd.DataFrame(tmp_label.groupby('timestamp_floor')['isAnomaly'].apply(lambda x: 1 in list(x)))

    confusion_matrix = pd.crosstab(label_eval['isAnomaly'], df_res_eval['value.is_anomaly'])
    if print_out:
        print(confusion_matrix)
        print(f'f1: {f1_score(label_eval, df_res_eval):.3f}, precision: {precision_score(label_eval, df_res_eval):.3f}, recall: {recall_score(label_eval, df_res_eval):.3f}')
    return confusion_matrix


### Section 2. Config specification and fetch AD results from ADX

In [ ]:
# Specify ADX Config
adx_database = "mvadbbdb"
adx_table = "<ADX_TABLE>"
adx_linked_service = "ADT_Data_History"

# Specify columns to plot
cols_to_include = ['dtmi_syntheticfactory_sourcemachine_1_A_water_flow', 
                    'dtmi_syntheticfactory_sourcemachine_1_A_oil_flow', 
                    'dtmi_syntheticfactory_sourcemachine_1_B_water_flow', 
                    'dtmi_syntheticfactory_sourcemachine_1_B_oil_flow']

# Specify time_window to plot
time_window = ['2022-06-01 00:00:00', '2022-07-01 00:00:00']

In [ ]:
# Fetch data with MVAD results from ADX
df = get_mvad_result_from_adx(
    cols_to_include=cols_to_include, time_window=time_window,
    adx_linked_service=adx_linked_service, adx_database=adx_database, adx_table=adx_table)

### Section 3. Make plots with or without labels

#### 3.1. Make Plots W/O Labels

In [ ]:
# Take a look at MVAD results and performance with default severity threshold to define an anomaly
min_severity = None
label = None

plot_mvad(df=df, cols_to_plot=cols_to_include, min_severity=min_severity, label=label)

In [ ]:
# A lower severity threshold will increase the likelihood of true anomalies capture, 
# with the risk of more false alarms.
min_severity = 0.1
label = None

plot_mvad(df=df, cols_to_plot=cols_to_include, min_severity=min_severity, label=label)

In [ ]:
# A higher severity threshold will help filter out more false alarms, 
# with the risk of missing true anomalies.
min_severity = 0.9
label = None

plot_mvad(df=df, cols_to_plot=cols_to_include, min_severity=min_severity, label=label)

#### 3.2. Make Plots W/ Labels

In [ ]:
# # Take a look at MVAD results and performance with default severity threshold to define an anomaly
# min_severity = None
# label = pd.read_csv(str(Path(os.getcwd()).parent.parent) + '\anomaly_label.csv')
# floor_bin = '1min'
# print(f'With Floor_bin = {floor_bin}, Severity_thres = {min_severity}')

# _ = evaluate(label, df.rename(columns={'isAnomaly': 'value.is_anomaly', 'severity': 'value.severity'}), floor_bin, min_severity)

# plot_mvad(df=df, cols_to_plot=cols_to_include, min_severity=min_severity, label=label)

In [ ]:
# # A lower severity threshold will increase the likelihood of true anomalies capture, 
# # with the risk of more false alarms.
# min_severity = 0.1
# label = pd.read_csv(str(Path(os.getcwd()).parent.parent) + '\anomaly_label.csv')
# floor_bin = '1min'
# print(f'With Floor_bin = {floor_bin}, Severity_thres = {min_severity}')

# _ = evaluate(label, df.rename(columns={'isAnomaly': 'value.is_anomaly', 'severity': 'value.severity'}), floor_bin, min_severity)

# plot_mvad(df=df, cols_to_plot=cols_to_include, min_severity=min_severity, label=label)

In [ ]:
# # A higher severity threshold will help filter out more false alarms, 
# # with the risk of missing true anomalies.
# min_severity = 0.9
# label = pd.read_csv(str(Path(os.getcwd()).parent.parent) + '\anomaly_label.csv')
# floor_bin = '1min'
# print(f'With Floor_bin = {floor_bin}, Severity_thres = {min_severity}')

# _ = evaluate(label, df.rename(columns={'isAnomaly': 'value.is_anomaly', 'severity': 'value.severity'}), floor_bin, min_severity)

# plot_mvad(df=df, cols_to_plot=cols_to_include, min_severity=min_severity, label=label)